# Hola Mundo: Desde el fichero a la pantalla

## ¿Quién soy yo?

* Jesús Espino 
  * Twitter: @jespinog
  * Github: http://github.com/jespino
  * Email: jespinog@gmail.com
* Desarrollador en:
  <img src="http://www.bokzuy.com/images/logo_kaleidos_big.png" width="200"> <img src="http://kaleidos.net/media/filer_public_thumbnails/filer_public/e5/14/e514887a-d582-44f6-9503-a892cd26a619/taiga-logo.jpg__300x300_q85_subsampling-2.jpg" width="200">

## ¿De que vamos a hablar?
* Compiladores
* Interpretes
* Python
* Hola Mundo

## ¿Qué es un compilador?
* Convierte un código fuente en codigo ejecutable por una maquina
* Normalmente se divide en varias fases (analisis lexico, analisis sintactico, construcción de arbol abstracto, generación de código maquina)

## ¿Qué es un interprete?
* Lee un código fuente, y lo interpreta directamente.
* Va leyendo y ejecutando el código tal cual.

## ¿Que es python?
* ¿Es interpretado?
* ¿Es compilado?

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />

## Ambas cosas

* Python compila el código a ByteCode
* El ByteCode es un código que interpreta la "Maquina Virtual" de python.

## Y ¿cómo lo hace?
![](compiler-graph.jpg)

## Hola Mundo: Del fichero a la pantalla

In [1]:
source_code = open("holamundo.py", "r").read()
print(source_code)

def holamundo(name="mundo"):
    print("Hola {}".format(name))

holamundo("test")



In [2]:
import holamundo
holamundo.holamundo()

Hola test
Hola mundo


## El resultado del compilador

* El compilador genera ByteCode, veamos como

In [3]:
c = compile(source_code, filename="holamundo.py", mode="exec")
print(type(c))
print(c.co_code)
print([b for b in bytes(c.co_code)])
print(c.co_code.hex())

<class 'code'>
b'd\x00\x00d\x01\x00d\x02\x00\x84\x01\x00Z\x00\x00e\x00\x00d\x03\x00\x83\x01\x00\x01d\x04\x00S'
[100, 0, 0, 100, 1, 0, 100, 2, 0, 132, 1, 0, 90, 0, 0, 101, 0, 0, 100, 3, 0, 131, 1, 0, 1, 100, 4, 0, 83]
6400006401006402008401005a00006500006403008301000164040053


## ¿Pero que demonios significa esto?

 * https://github.com/python/cpython/blob/3.6/Include/opcode.h
 * https://github.com/python/cpython/blob/3.6/Python/ceval.c#L1220

## ¿En serio? ¿Código fuente?

* Vaaaale... si lo prefieres tambien hay documentación:
  * https://docs.python.org/3.6/library/dis.html#python-bytecode-instructions

## Vamos a leerlo un poco mejor

In [4]:
import dis
dis.dis(c)

  1           0 LOAD_CONST               0 ('mundo')
              3 LOAD_CONST               1 (<code object holamundo at 0x7f2b742d6a50, file "holamundo.py", line 1>)
              6 LOAD_CONST               2 ('holamundo')
              9 MAKE_FUNCTION            1
             12 STORE_NAME               0 (holamundo)

  4          15 LOAD_NAME                0 (holamundo)
             18 LOAD_CONST               3 ('test')
             21 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             24 POP_TOP
             25 LOAD_CONST               4 (None)
             28 RETURN_VALUE


In [5]:
print(type(holamundo.holamundo))
print(type(holamundo.holamundo.__code__))
print(holamundo.holamundo.__code__.co_code)
print([b for b in bytes(holamundo.holamundo.__code__.co_code)])
print(holamundo.holamundo.__code__.co_code.hex())

<class 'function'>
<class 'code'>
b't\x00\x00d\x01\x00j\x01\x00|\x00\x00\x83\x01\x00\x83\x01\x00\x01d\x00\x00S'
[116, 0, 0, 100, 1, 0, 106, 1, 0, 124, 0, 0, 131, 1, 0, 131, 1, 0, 1, 100, 0, 0, 83]
7400006401006a01007c00008301008301000164000053


In [6]:
dis.dis(holamundo.holamundo)

  2           0 LOAD_GLOBAL              0 (print)
              3 LOAD_CONST               1 ('Hola {}')
              6 LOAD_ATTR                1 (format)
              9 LOAD_FAST                0 (name)
             12 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             15 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             18 POP_TOP
             19 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## Y ¿Como lo ejecuto?

* Usando `exec` o `eval`
* `exec` ejecuta un bloque de código y no devuelve resultado
* `eval` ejecuta una expresion y devuelve resultado

In [9]:
exec(c)
holamundo.holamundo()

Hola test


AttributeError: 'function' object has no attribute 'holamundo'

## Pero ¿cómo llego hasta aquí?

* El compilado se divide principalmente en 5 fases:
    * Tokenizado.
    * Construcción del arbol sintaction (parseado).
    * Construcción del arbol sintaction abstracto (AST).
    * Construcción de la tabla de simbolos.
    * Traducción a ByteCode.

## El primer paso: Tokenizado

El primer paso es el tokenizado del codigo, que divide y etiqueta cada uno de los elementos de nuestro codigo para luego ser procesados.

In [ ]:
import tokenize

tokens = tokenize.tokenize(open('holamundo.py', "rb").readline)
print("\n".join([str(token) for token in tokens]))

### ¿Donde ocurre la magia?

* https://github.com/python/cpython/blob/3.6/Parser/tokenizer.c#L1363

* pgen genera el código necesario para la gramatica
* asdl_c.py genera el codigo necesario para la generación del AST

## Segundo paso: Analisis sintactico
Tras el tokenizado se realiza el analisis sintactico, que organiza nuestros tokens en un Syntax-Tree

In [ ]:
import parser
st = parser.suite(source_code)
st.totuple()

Podemos verlo un poco mejor si traducimos los IDS a Tokens y Simbolos

In [ ]:
import stpp
stpp.stpp(st)

In [ ]:
mod = st.compile()
exec(mod)
holamundo()

### ¿Donde ocurre la magia?

* pgen genera el código necesario para la gramatica:
  * https://github.com/python/cpython/blob/3.6/Grammar/Grammar
* https://github.com/python/cpython/blob/3.6/Parser/parsetok.c#L183

## Tercer paso: Analisis semantico
Tras la construcción del analisis sintactico se hace el analisis semantico y se construye un Abstract-Syntax-Tree

In [17]:
import ast
x = ast.parse(source_code)
rint(ast.dump(x))

Module(body=[FunctionDef(name='holamundo', args=arguments(args=[arg(arg='name', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[Str(s='mundo')]), body=[Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Call(func=Attribute(value=Str(s='Hola {}'), attr='format', ctx=Load()), args=[Name(id='name', ctx=Load())], keywords=[])], keywords=[]))], decorator_list=[], returns=None), Expr(value=Call(func=Name(id='holamundo', ctx=Load()), args=[Str(s='test')], keywords=[]))])


In [16]:
import astpp
print(astpp.dump(ast.parse(source_code)))

Module(body=[
    FunctionDef(name='holamundo', args=arguments(args=[
        arg(arg='name', annotation=None),
      ], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[
        Str(s='mundo'),
      ]), body=[
        Expr(value=Call(func=Name(id='print', ctx=Load()), args=[
            Call(func=Attribute(value=Str(s='Hola {}'), attr='format', ctx=Load()), args=[
                Name(id='name', ctx=Load()),
              ], keywords=[]),
          ], keywords=[])),
      ], decorator_list=[], returns=None),
    Expr(value=Call(func=Name(id='holamundo', ctx=Load()), args=[
        Str(s='test'),
      ], keywords=[])),
  ])
[<_ast.FunctionDef object at 0x7f2b74200278>, <_ast.Expr object at 0x7f2b74277940>]


In [8]:
import symtable
import symtablepp
table = symtable.symtable(source_code, filename="holamundo.py", compile_type="exec")
symtablepp.symtablepp(table)

 <SymbolTable for top in holamundo.py>
   <symbol 'holamundo'>
   <Function SymbolTable for holamundo in holamundo.py>
     <symbol 'name'>
     <symbol 'print'>


### ¿Donde ocurre la magia?

* asdl_c.py genera el codigo necesario para la generación del AST:
  * https://github.com/python/cpython/blob/3.6/Parser/Python.asdl
* https://github.com/python/cpython/blob/3.6/Python/ast.c#L761

## Resumen
![](compiler-graph.jpg)

## Dudas